#### How to stream the data from a single camera to the screen with 6 different types of cameras

In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2

In [2]:
client=carla.Client('localhost',2000)

In [3]:
world=client.get_world()

In [4]:
bp_lib=world.get_blueprint_library()
spawn_points=world.get_map().get_spawn_points()

In [5]:
# print(len(spawn_points))
# print(spawn_points[79])

In [6]:
vehicle_bp=bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle=world.try_spawn_actor(vehicle_bp, spawn_points[79])
# vehicle=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator=world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

# spawn_actor(self, blueprint, transform, attach_to=None, attachment=Rigid) : 
    shows some sensors being attached to a car when spawned

In [7]:
for i in range(50):
    vehicle_bp=random.choice(bp_lib.filter('vehicle'))
    npc=world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [8]:
# set traffic in motion

for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

# spawn_actor(self, blueprint, transform, attach_to=None, attachment=Rigid) : 
    shows some sensors being attached to a car when spawned

In [9]:
# set initial camera translation
camera_init_trans=carla.Transform(carla.Location(z=2))

# Add one of each type of cameras
camera_bp=bp_lib.find('sensor.camera.rgb')
camera=world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle) 

sem_camera_bp=bp_lib.find('sensor.camera.semantic_segmentation')
sem_camera=world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

inst_camera_bp=bp_lib.find('sensor.camera.instance_segmentation')
inst_camera=world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle)

depth_camera_bp=bp_lib.find('sensor.camera.depth')
depth_camera=world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)

dvs_camera_bp=bp_lib.find('sensor.camera.dvs')
dvs_camera=world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=vehicle)

opt_camera_bp=bp_lib.find('sensor.camera.optical_flow')
opt_camera=world.spawn_actor(opt_camera_bp, camera_init_trans, attach_to=vehicle)

In [10]:
# define respective callbacks

#(overall rows, inside rows, columns)
def rgb_callback(image, data_dict):
    data_dict['rgb_image']=np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

def inst_callback(image, data_dict):
    data_dict['inst_image']=np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def dvs_callback(data, data_dict):
    dvs_events=np.frombuffer(data.raw_data, dtype=np.dtype([('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', bool)])) #why uint16, 64???
    data_dict['dvs_image']=np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img=np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'],dvs_events[:]['x'],dvs_events[:]['pol']*2]=255
    data_dict['dvs_image'][:,:,0:3]=dvs_img

# CityScapesPalette :
    semantic camera gives a different integer for each different class of object
    the integers are between 0 and 22
    in order to visualize it,use CityScapesPalette

In [11]:
def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image']=np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# LogarithmicDepth :
    Converts the image to a depth map using a logarithmic scale, leading to better precision for small distances at the expense of losing it when further away

In [12]:
def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image']=np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# get_color_coded_flow(self) :
    Visualization helper. Converts the optical flow image to an RGB image.
    Return: carla.Image

In [13]:
# extract the data from the camera and then reshape

def opt_callback(data, data_dict):
    image=data.get_color_coded_flow()
    img=np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img[:,:,3]=255 #hm, why???.0  
    data_dict['opt_image']=img

In [14]:
#initialize parameters and data

image_w=camera_bp.get_attribute('image_size_x').as_int()
image_h=camera_bp.get_attribute('image_size_y').as_int()

sensor_data={'rgb_image':np.zeros((image_h, image_w, 4)),
            'sem_image':np.zeros((image_h, image_w, 4)),
            'depth_image':np.zeros((image_h, image_w, 4)),
            'dvs_image':np.zeros((image_h, image_w, 4)),
             'opt_image': np.zeros((image_h, image_w, 4)),
             'inst_image':np.zeros((image_h, image_w, 4))}

# computer vision
    the way of teaching intelligence to machine and making them see things like humans. What allows computers to see and process visual data like humans, analyzing images to produce useful information

# OpenCV
    vision library of programming functions mainly aimed at real time computer vision. it can be showed a digital image seen by a computer in matrix
    
# namedWindow() :
    is used to create a window with a suitable name and size to display images and videos on the screen.
    
# WINDOW_NORMAL : 
    it enables you to resize the window
   
# WINDOW_AUTOSIZE: 
    it adjusts automatically the window size to fit the displayed image (see imshow ), and you cannot change the window size manually

In [15]:
# OpenCV named window for display

cv2.namedWindow('All cameras', cv2.WINDOW_AUTOSIZE)

In [16]:
# tile all data in one array

top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']), axis=1)
tiled = np.concatenate((top_row, lower_row), axis=0)

# print(tiled)
print(tiled.shape)

(1200, 2400, 4)


In [17]:
# using cv2.imshow() to display the image
cv2.imshow('All cameras', tiled)

# Waiting 1ms for user to press any key
cv2.waitKey(1)

-1

In [18]:
# Set sensors recording

camera.listen(lambda image:rgb_callback(image, sensor_data))
sem_camera.listen(lambda image:sem_callback(image, sensor_data))
inst_camera.listen(lambda image:inst_callback(image, sensor_data))
depth_camera.listen(lambda image:depth_callback(image, sensor_data))
dvs_camera.listen(lambda image:dvs_callback(image, sensor_data))
opt_camera.listen(lambda image:opt_callback(image, sensor_data))

In [19]:
# while True:
#     if cv2.waitKeay(1)==ord('q'):
#         break

In [20]:
# camera.stop()
# sem_camera.stop()
# inst_camera.stop()
# depth_camera.stop()
# dvs_camera.stop()
# opt_camera.stop()
# cv2.destroyAllWindows()

In [21]:
while True:
    # Tile camera images into one array
    top_row=np.concatenate((sensor_data['rgb_image'],sensor_data['sem_image'],sensor_data['inst_image']), axis=1)
    lower_row=np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']),axis=1)
    tiled=np.concatenate((top_row, lower_row), axis=0)
    
#     print(top_row.shape)
#     print(top_row[0])
    
#     print(lower_row.shape)
#     print(lower_row[0])
    
#     print(tiled.shape)
#     print(tiled[0])
    
    cv2.imshow('All cameras', tiled)
    
    if cv2.waitKey(1)==ord('q'):
        break 

(600, 2400, 4)
[[ 69  85  99 255]
 [ 69  85  99 255]
 [ 69  83  98 255]
 ...
 [138 229   3 255]
 [138 229   3 255]
 [138 229   3 255]]
(600, 2400, 4)
[[ 75.  75.  75. 255.]
 [ 75.  75.  75. 255.]
 [ 75.  75.  75. 255.]
 ...
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]]
(1200, 2400, 4)
[[ 69.  85.  99. 255.]
 [ 69.  85.  99. 255.]
 [ 69.  83.  98. 255.]
 ...
 [138. 229.   3. 255.]
 [138. 229.   3. 255.]
 [138. 229.   3. 255.]]
(600, 2400, 4)
[[ 69  85  99 255]
 [ 69  85  99 255]
 [ 69  83  98 255]
 ...
 [138 229   3 255]
 [138 229   3 255]
 [138 229   3 255]]
(600, 2400, 4)
[[ 75  75  75 255]
 [ 75  75  75 255]
 [ 75  75  75 255]
 ...
 [  0   0   1 255]
 [  0   0   1 255]
 [  0   0   1 255]]
(1200, 2400, 4)
[[ 69  85  99 255]
 [ 69  85  99 255]
 [ 69  83  98 255]
 ...
 [138 229   3 255]
 [138 229   3 255]
 [138 229   3 255]]
(600, 2400, 4)
[[ 69  85  98 255]
 [ 69  85  99 255]
 [ 66  85  98 255]
 ...
 [138 229   3 255]
 [138 229   3 255]
 [138 229   3 255]]
(600,

In [22]:
camera.stop()
sem_camera.stop()
inst_camera.stop()
depth_camera.stop()
dvs_camera.stop()
opt_camera.stop()
cv2.destroyAllWindows()